In [29]:
import pandas as pd
import requests
import time

In [30]:
api_key = 'RGAPI-0b437417-9c5d-462b-8402-c3e3cc8329c2'
summoner_name = '학장주유소'

In [31]:
requesturl = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/'+summoner_name+\
'?api_key='+api_key
requests.get(requesturl).json()

{'id': 'FWJXIFLSt6WtPPuZ_y9UeLHTdnRqj_jwo4XQFkWQkIQth1U',
 'accountId': 'D6u62CpEWag_XxSzT-bEQF5-T7d2mtZUAyxXMT6W9_e9Ais',
 'puuid': '2CIbiJ46u2meb-qaLoVFC8QTN5vU26PWRRkLBN5n_SFnBRaB_SarMMAqARqRRxTE_Ju9CthevWrzoA',
 'name': '학장주유소',
 'profileIconId': 5766,
 'revisionDate': 1692380221000,
 'summonerLevel': 251}

In [32]:
summoner_puuid = requests.get(requesturl).json()['puuid']

In [37]:
games_list = []
index = 0
while True:
    requesturl = 'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/'+summoner_puuid+'/ids?queue=420&start='+str(index)+'&count=10&api_key='+api_key
    
    while True:
        response = requests.get(requesturl)
        if response.ok:
            game_list_new = response.json()
            break
    
    games_list.extend(game_list_new)
    
    if len(game_list_new) != 100:
        break
        
    index += 100
    time.sleep(1)

In [38]:
games_list

['KR_6597443448',
 'KR_6597407402',
 'KR_6597371976',
 'KR_6585775901',
 'KR_6585729803',
 'KR_6576920126',
 'KR_6576857423',
 'KR_6576814681',
 'KR_6569720853',
 'KR_6569656755']

In [39]:
games_df = pd.DataFrame()
for matchId in games_list:

    requesturl = 'https://asia.api.riotgames.com/lol/match/v5/matches/'+matchId+'?api_key='+api_key
    
    while True:
        response = requests.get(requesturl)
        if response.ok:
            game_json = response.json()
            break

    game_df = pd.DataFrame(game_json['info']['participants'])[['puuid','championId','win','kills','deaths','assists']]
    game_df = game_df[game_df.puuid == summoner_puuid][['championId','win','kills','deaths','assists']]
        
    games_df = pd.concat([games_df,game_df])
    time.sleep(1)
    

In [40]:
pd.DataFrame(game_json['info']['participants'])[['puuid','championId','win','kills','deaths','assists']]

,puuid,championId,win,kills,deaths,assists
0,deN9hr7Z0JiG61N23EnRy2hJ6LnVeTrnYwSD7PkC7h0iP2...,777,False,5,8,4
1,lwFwlZMCMiVP9nQ3LwAWaw_S8bVz-QLva9ifh-oF2Wy9ae...,427,False,2,4,11
2,IFZWWXcBeMqKtotwCtOQEeAn3zSAriT6mlqPSDLIcqEmm9...,69,False,4,3,1
3,2CIbiJ46u2meb-qaLoVFC8QTN5vU26PWRRkLBN5n_SFnBR...,202,False,11,4,5
4,xKtXu3CgfjEwcdJYZg3zmlYPqnprA4nFC5w7-9REa8QEy2...,99,False,2,9,13
5,6Xn0Q8qGu2ysxBcuhoy4vaci8B92VKPYMx_LVXGmkHhnwy...,126,True,7,5,4
6,uH8dMBH6Coy6g3zSn4DDJ3EgqyVaD8EhMtOXfTVnX9kUm1...,11,True,11,3,5
7,S7Vvh7ncZS-Zz1tv_VCuhhZ-H7q5rAhlp8fTYA9UMYItA9...,103,True,3,5,6
8,mCNDU1BO1kjhdVpb8nmr-fd3wEKAEuY_RD5dMkasCSqpcg...,81,True,6,8,4
9,4lmjBdeO5ogS482S5JZZOeiaABvB8QSwgYr2rf3pHQykUK...,43,True,1,4,11


In [41]:
games_df = games_df.groupby(['championId'],as_index=False).\
apply(lambda x: pd.Series({'win_count':sum(x.win),
                           'count':len(x),
                           'kills':sum(x.kills),
                           'deaths':sum(x.deaths),
                           'assists':sum(x.assists)
                          }))

In [42]:
last_version = requests.get("https://ddragon.leagueoflegends.com/api/versions.json").json()[0]
champion_constant = requests.get("http://ddragon.leagueoflegends.com/cdn/"+last_version+"/data/ko_KR/champion.json")

In [43]:
# Json 파일을 DataFrame으로 변환
champion_df = pd.DataFrame(champion_constant.json()['data']).T[['key','name']]

# 변수형 문자 -> 숫자
champion_df['key'] = pd.to_numeric(champion_df['key'])

In [44]:
games_df = games_df.merge(champion_df,
                          how = 'left',
                          left_on = 'championId',
                          right_on = 'key'
                         )[['name','count','win_count','kills','deaths','assists']]

In [45]:
games_df['WR'] = games_df['win_count'] / games_df['count']
games_df['KDA'] = (games_df['kills']+games_df['assists']) / games_df['deaths']

In [46]:
games_df.sort_values('count',ascending = False)[['name','WR','count','KDA']]

,name,WR,count,KDA
5,라칸,0.666667,3,2.777778
4,비에고,0.500000,2,4.000000
0,그레이브즈,1.000000,1,11.500000
1,노틸러스,1.000000,1,2.833333
2,케인,1.000000,1,2.416667
3,진,0.000000,1,4.000000
6,자야,0.000000,1,0.500000


In [ ]:
# 내가 시간대별로 아이템 뭐샀는지 한번 해보자